# 5. MGWR

## 5.4 MGWRをコードで理解する

###5.4.1 準備

In [1]:
from google.colab import drive
drive.mount('/content/mount')

Mounted at /content/mount


In [2]:
%cd '/content/mount/MyDrive/SDS'

/content/mount/MyDrive/SDS


In [3]:
!pip install -q libpysal
!pip install -q geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import libpysal as ps

# 自作モジュールのインポート
from Functions.utils import *
from Functions.commons import *
from Functions.gwr import GWR
from Functions.sel_bw import Sel_BW

In [5]:
georgia = gpd.read_file(ps.examples.get_path('G_utm.shp'))

In [6]:
y, X, coords = get_input_data(georgia['PctBach'], georgia[['PctFB', 'PctBlack', 'PctRural']], georgia[['X', 'Y']], std=True)

###5.4.2 最適バンド幅の選択：SelBW_MGWRクラスの作成

####5.4.2.1 クラスの定義とコンストラクタ

In [ ]:
class Sel_BW_MGWR(Sel_BW):  # Sel_BWクラスの継承
  '''
  入力は被説明変数y, 説明変数X, 緯度経度coords
  '''

  # コンストラクタ
  def __init__(self, y, X, coords):
    super().__init__(y, X, coords)  # 親クラス(Sel_BWクラス)のメンバ変数の継承

In [ ]:
selector = Sel_BW_MGWR(y, X, coords)
opt_bw = selector.search()
print(opt_bw)

117


####5.4.2.2 初期値の設定

In [ ]:
class Sel_BW_MGWR(Sel_BW):  # Sel_BWクラスの継承
  '''
  入力は被説明変数y, 説明変数X, 緯度経度coords
  '''

  # コンストラクタ
  def __init__(self, y, X, coords):
    super().__init__(y, X, coords)  # 親クラス(Sel_BWクラス)のメンバ変数の継承


  # GWRに基づく初期値を計算する関数
  def get_init_values(self):
    # 変数の宣言
    y = self.y
    X = self.X
    N = self.N
    K = self.K

    # GWRによる最適バンド幅を求める
    gwr_bw = self.search()
    # 重み行列のリストの作成(返されたバンド幅 - 1)
    w_list = get_weight_list(N, self.sorted_dist_list, self.reverse_dist_list_idx, gwr_bw - 1)

    # 受け渡す変数の作成
    f_list = []
    e = np.zeros(shape=(N, 1))
    R_list = []
    S = np.zeros(shape=(N, N))

    for j in range(K):
      R_j = np.zeros(shape=(N, N))  # R_jの作成
      for i in range(N):
        # 重みが対角要素に並んだ「重み行列」の作成
        W_i = get_weight_matrix(w_list[i])
        # 地点iのr_ijを求める
        e_j =  np.matrix(np.eye(K)[j])   # One-Hotベクトルの作成
        R_j[i, :] = np.matrix(X[i, j]) @ e_j @ np.linalg.inv(X.T @ W_i @ X) @ X.T @ W_i

      f_j = R_j @ y  # f_jの推定値を計算
      f_list.append(f_j)  # f_jの格納
      R_list.append(R_j)  # R_jの格納

    for j in range(K):
      S = S + R_list[j]  # Sの計算

    e = y - S @ y  # eの計算

    return S, e, R_list, f_list

In [ ]:
selector = Sel_BW_MGWR(y, X, coords)
S, e, R_list, f_list = selector.get_init_values()

In [ ]:
print(S.shape)
print(np.round(np.trace(S),3))

(159, 159)
11.805


In [ ]:
print(e.shape)
print(e.T @ e)
print(e[:10])

(159, 1)
[[51.1861903]]
[[-0.05336116]
 [-0.12358663]
 [-0.45794867]
 [ 0.40970968]
 [-0.04869648]
 [-0.28467294]
 [-0.49699411]
 [-0.28719305]
 [-0.52520405]
 [-0.29526495]]


In [ ]:
print(len(R_list))
print(R_list[0].shape)
print(R_list[0])

4
(159, 159)
[[2.22941917e-02 2.53213897e-02 2.01662463e-02 ... 6.77848977e-04
  7.62644916e-03 9.12390228e-03]
 [2.07119408e-02 2.56232623e-02 2.21309597e-02 ... 3.67269978e-05
  5.73751183e-03 1.42206411e-02]
 [2.17374355e-02 2.56838739e-02 2.07047012e-02 ... 4.04353125e-04
  6.92170685e-03 1.06620418e-02]
 ...
 [3.80611620e-04 0.00000000e+00 0.00000000e+00 ... 2.53672610e-02
  1.15184530e-02 0.00000000e+00]
 [8.06651602e-03 2.70517562e-03 5.68079167e-03 ... 4.28187378e-03
  1.94279420e-02 2.91902085e-03]
 [1.59641444e-02 2.24051672e-02 2.06584725e-02 ... 0.00000000e+00
  5.81420003e-03 2.36570818e-02]]


In [ ]:
print(len(f_list))
print(f_list[0].shape)
print(f_list[0][:10])

4
(159, 1)
[[-0.23204581]
 [-0.27922382]
 [-0.24894402]
 [-0.23036771]
 [ 0.19066198]
 [ 0.16316332]
 [ 0.16606588]
 [ 0.12335741]
 [-0.2692158 ]
 [-0.28700589]]


####5.4.2.3 イタレーション(繰り返し計算)

In [7]:
class Sel_BW_MGWR(Sel_BW):  # Sel_BWクラスの継承
  '''
  入力は被説明変数y, 説明変数X, 緯度経度coords
  '''

  # コンストラクタ
  def __init__(self, y, X, coords):
    super().__init__(y, X, coords)  # 親クラス(Sel_BWクラス)のメンバ変数の継承


  # GWRに基づく初期値を計算する関数
  def get_init_values(self):
    # 変数の宣言
    y = self.y
    X = self.X
    N = self.N
    K = self.K

    # GWRによる最適バンド幅を求める
    gwr_bw = self.search()
    # 重み行列のリストの作成(返されたバンド幅 - 1)
    w_list = get_weight_list(N, self.sorted_dist_list, self.reverse_dist_list_idx, gwr_bw - 1)

    # 受け渡す変数の作成
    f_list = []
    e = np.zeros(shape=(N, 1))
    R_list = []
    S = np.zeros(shape=(N, N))

    for j in range(K):
      R_j = np.zeros(shape=(N, N))  # R_jの作成
      for i in range(N):
        # 重みが対角要素に並んだ「重み行列」の作成
        W_i = get_weight_matrix(w_list[i])
        # 地点iのr_ijを求める
        e_j =  np.matrix(np.eye(K)[j])   # One-Hotベクトルの作成
        R_j[i, :] = np.matrix(X[i, j]) @ e_j @ np.linalg.inv(X.T @ W_i @ X) @ X.T @ W_i

      f_j = R_j @ y  # f_jの推定値を計算
      f_list.append(f_j)  # f_jの格納
      R_list.append(R_j)  # R_jの格納

    for j in range(K):
      S = S + R_list[j]  # Sの計算

    e = y - S @ y  # eの計算

    return S, e, R_list, f_list


  # 最適バンド幅を求める関数
  def mgwr_search(self):
    # 変数の宣言
    y = self.y
    X = self.X
    N = self.N
    K = self.K

    # イタレーションの初期値を求める
    S, e, R_list, f_list = self.get_init_values()

    # 設定
    SOC_f = 1e+5  # SOC_fの初期値
    threshold = 1e-5  # 閾値
    diff_f = np.zeros(K)  # SOC_fの根号内部の分子部分の差の初期値
    mgwr_bw = np.zeros(K).astype(int)  # 最適バンド幅を格納する入れ物

    iter = 1  # イタレーションのためのカウンタ(初期値で1を代入)
    # イタレーション
    while SOC_f > threshold:
      print('iteration ' + str(iter))  # イタレーションの回数表示
      for j in range(K):
        # 単変量GWRのためのSel_BWクラスオブジェクトを作成
        tmp_y = f_list[j] + e
        tmp_X = X[:, j]
        gwr_j = Sel_BW(tmp_y, tmp_X, coords)
        # searchメソッドにより最適バンド幅を求める
        mgwr_bw[j] = gwr_j.search()
        # mgwr_bwに基づく重みリストの作成
        w_list = get_weight_list(N, gwr_j.sorted_dist_list, gwr_j.reverse_dist_list_idx, mgwr_bw[j] - 1)
        # 加法項のバンド幅の表示
        print('additive term '+ str(j) + ' : ' + str(mgwr_bw[j]))

        A_j = np.zeros(shape=(N, N))  # A_jの作成
        for i in range(N):
          # 重みが対角要素に並んだ「重み行列」の作成
          W_i = get_weight_matrix(w_list[i])
          # A_jを求める
          A_j[i,:] = np.matrix(X[i,j]) @ np.linalg.inv(tmp_X.T @ W_i @ tmp_X) @ tmp_X.T @ W_i

        R_j = A_j @ R_list[j] + A_j - A_j @ S  # R_jの更新
        S = S - R_list[j] + R_j  # Sの更新
        R_list[j] = R_j  # R_jの格納
        f_j = R_j @ y  #f_jの更新
        diff_f[j] = ((f_j - f_list[j]).T @ (f_j - f_list[j]))[0,0] / N  # diff_f
        f_list[j] = f_j  # 更新されたf_jを格納

      e = y - S @ y  # eの更新

      denom = np.sum(f_list, axis=1)**2  # SOC_fの根号内部の分母の一部
      SOC_f = np.sqrt(np.sum(diff_f) / np.sum(denom))  # SOC_fの計算

      iter += 1  # カウンタの更新

    return [S, e, R_list, f_list, mgwr_bw]

In [8]:
selector = Sel_BW_MGWR(y, X, coords)

In [9]:
opt_list = selector.mgwr_search()  # おおよそ20～30秒で収束

iteration 1
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158
iteration 2
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158
iteration 3
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158
iteration 4
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158
iteration 5
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158
iteration 6
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158
iteration 7
additive term 0 : 92
additive term 1 : 101
additive term 2 : 135
additive term 3 : 158


In [10]:
# 最適バンド幅の確認
opt_list[4]

array([ 92, 101, 135, 158])

### 5.4.3 MGWRの推定の実行

####5.4.3.1 大域的推定

In [11]:
class MGWR(GWR):  # GWRクラスを継承
  '''
    入力は被説明変数y, 説明変数X, 緯度経度coords,
    Sel_BW_MGWRで求めたリスト(opt_list)
  '''

  # コンストラクタ
  def __init__(self, y, X, coords, opt_list):
    super().__init__(y, X, coords, bw=0)  # MGWRではbwは使用しないので0とする
    # メンバ変数の追加
    self.S = opt_list[0]
    self.e = opt_list[1]
    self.R_list = opt_list[2]
    self.f_list = opt_list[3]
    self.bws = opt_list[4]


  # パラメータの推定
  def get_beta_hat(self):
    # 変数の宣言
    y = self.y
    X = self.X
    N = self.N
    K = self.K
    e = self.e
    f_list = self.f_list
    bws = self.bws

    fij_mat = np.matrix(np.zeros_like(X))  # fの入れ物を作る
    for j in range(K):
      # 重みリストの作成
      w_list = get_weight_list(N, self.sorted_dist_list, self.reverse_dist_list_idx, bws[j]-1)
      # 各加法項の変数とそれ対応する被説明変数の設定
      tmp_y = f_list[j] + e   #fj_hat_list[j] + e
      tmp_X = X[:, j].reshape(-1,1)

      A_j = np.zeros(shape=(N, N))  # A_jの入れ物を作る
      for i in range(N):
        W_i = get_weight_matrix(w_list[i])  # 重み行列の作成
        A_j[i, :] = X[i, j] * np.linalg.inv(tmp_X.T @ W_i @ tmp_X) @ tmp_X.T @ W_i

      # fに結果を挿入
      fij_mat[:, j] = A_j @ tmp_y

    # 要素ごとの割り算でパラメータ推定値を求める
    beta_hat = fij_mat / X

    return beta_hat

  # SとRのリストを求める関数
  def get_S_Rlist(self):
    # 変数の宣言
    y = self.y
    X = self.X
    N = self.N
    K = self.K
    bws = self.bws

    # R_listとSの入れ物を作る
    R_list = []
    S = np.zeros(shape=(N, N))

    for j in range(K):
      # R_jの入れ物
      R_j = np.zeros(shape=(N, N))
      # 重みリストの作成
      w_list = get_weight_list(N, self.sorted_dist_list, self.reverse_dist_list_idx, bws[j]-1)
      # One-Hotベクトルの作成
      e_j = np.matrix(np.eye(K)[j])

      for i in range(N):
        # 重み行列の作成
        W_i = get_weight_matrix(w_list[i])
        # 地点iのr_ijを求める
        R_j[i,:] = X[i, j] * e_j @ np.linalg.inv(X.T @ W_i @ X) @ X.T @ W_i

      # R_listにR_jを逐次追加
      R_list.append(R_j)
      # Sを求める
      S = S + R_j

    return S, R_list

  def fit(self):
    self.beta_hat = self.get_beta_hat()
    self.S, self.R_list = self.get_S_Rlist()
    self.v1 = get_v1(self.S)
    self.df = self.N - self.v1
    self.y_hat = get_y_hat(self.y, self.S)
    self.resid = get_resid(self.y, self.y_hat)
    self.rss = get_rss(self.resid)
    self.sigma2 = get_sigma2(self.rss, self.N, self.v1)
    self.sigma_hat = np.sqrt(self.sigma2)
    self.mll = get_mll(self.y, self.y_hat, self.rss, self.N)
    self.aic = get_aic(self.mll, self.v1+1)
    self.aicc = get_aicc(self.mll, self.N, self.v1+1)
    self.bic = get_bic(self.mll, self.N, self.v1+1)
    self.R2, self.adjR2 = get_R2(self.y, self.rss, self.N, self.v1+1)

In [12]:
res = MGWR(y, X, coords, opt_list)
res.fit()

In [13]:
print(res.beta_hat)

[[-0.1970638   0.29443613 -0.04368603 -0.33114036]
 [-0.2533255   0.22802855 -0.02525266 -0.32520526]
 [-0.21962171  0.27948198 -0.03475291 -0.33018639]
 [-0.16793202  0.14551522 -0.02680253 -0.28670191]
 [ 0.16569777  0.67900968 -0.14024128 -0.29549926]
 [ 0.22241513  0.71752905 -0.07692143 -0.30481515]
 [ 0.23914978  0.72060953 -0.08214396 -0.3022401 ]
 [ 0.12810627  0.70619933 -0.04530557 -0.2970555 ]
 [-0.2226214   0.28820005 -0.03968639 -0.31411448]
 [-0.25149265  0.19220429 -0.02434425 -0.31824888]
 [ 0.10160408  0.54847271 -0.11769664 -0.28623597]
 [-0.03363332  0.39725974 -0.09231784 -0.29818571]
 [-0.2129971   0.26145015 -0.02617861 -0.33910245]
 [-0.24985578  0.14078903 -0.0173096  -0.31486248]
 [-0.12834496  0.32384125 -0.06752544 -0.3428091 ]
 [-0.07785883  0.3782631  -0.0947459  -0.3363983 ]
 [ 0.06626918  0.55548806 -0.13393797 -0.32454499]
 [ 0.20724478  0.67044997 -0.11147506 -0.29287553]
 [-0.13952347  0.16258203 -0.03302952 -0.27883134]
 [-0.21061287  0.25336374 -0.02

#### 5.4.3.2 局所的推定

In [14]:
class MGWR(GWR):  # GWRクラスを継承
  '''
    入力は被説明変数y, 説明変数X, Sel_BWで求めた最適バンド幅のリスト(np.ndarray型)
  '''

  # コンストラクタ
  def __init__(self, y, X, coords, opt_list):
    super().__init__(y, X, coords, bw=0)  # MGWRではbwは使用しないので0としておく
    # メンバ変数の追加
    self.S = opt_list[0]
    self.e = opt_list[1]
    self.R_list = opt_list[2]
    self.f_list = opt_list[3]
    self.bws = opt_list[4]
    self.local_std = None  # 追加部分
    self.local_tval = None  # 追加部分
    self.adj_alpha = None  # 追加部分
    self.adj_tval = None  # 追加部分


  # パラメータの推定
  def get_beta_hat(self):
    y = self.y
    X = self.X
    N = self.N
    K = self.K
    e = self.e
    f_list = self.f_list
    bws = self.bws

    fij_mat = np.matrix(np.zeros_like(X))
    for j in range(K):
      w_list = get_weight_list(N, self.sorted_dist_list, self.reverse_dist_list_idx, bws[j]-1)
      tmp_y = f_list[j] + e   #fj_hat_list[j] + e
      tmp_X = X[:, j].reshape(-1,1)

      A_j = np.zeros(shape=(N, N))
      for i in range(N):
        W_i = get_weight_matrix(w_list[i])
        A_j[i, :] = X[i, j] * np.linalg.inv(tmp_X.T @ W_i @ tmp_X) @ tmp_X.T @ W_i

      fij_mat[:, j] = A_j @ tmp_y

    beta_hat = fij_mat / X

    return beta_hat


  def get_S_Rlist(self):
    y = self.y
    X = self.X
    N = self.N
    K = self.K
    bws = self.bws

    R_list = []
    S = np.zeros(shape=(N, N))

    for j in range(K):
      R_j = np.zeros(shape=(N, N))  # R_jの作成
      w_list = get_weight_list(N, self.sorted_dist_list, self.reverse_dist_list_idx, bws[j]-1)
      e_j = np.matrix(np.eye(K)[j])  # One-Hotベクトルの作成

      for i in range(N):
        # 重みが対角要素に並んだ「重み行列」の作成
        W_i = get_weight_matrix(w_list[i])
        # 地点iのr_ijを求める
        R_j[i,:] = X[i, j] * e_j @ np.linalg.inv(X.T @ W_i @ X) @ X.T @ W_i

      R_list.append(R_j)
      S = S + R_j

    return S, R_list


  # MGWRにおける追加メソッド
  def get_adj_alpha(self):
    K = self.K
    R_list = self.R_list

    xi = 0.05  # 有意水準は0.05
    adj_alpha = []

    for k in range(K):
      enp_j = np.sum(np.diag(R_list[k]))
      adj_alpha.append(xi / enp_j)

    return adj_alpha


  def get_adj_tval(self):
    N = self.N
    K = self.K
    adj_alpha = self.adj_alpha

    adj_tval = []

    for k in range(K):
      # 修正された5％有意の臨界値をt分布から求める
      adj_tval.append(st.t.ppf(1-adj_alpha[k]/2, N-1))  # t分布の自由度は(データ数-共変量数)

    return adj_tval


  def get_local_std_and_tval(self):
    X = self.X
    N = self.N
    K = self.K
    bws = self.bws
    beta_hat = self.beta_hat
    S = self.S
    R_list = self.R_list
    sigma_hat = self.sigma_hat

    # 標準誤差stdおよびt値tvalの格納
    local_std = np.matrix(np.zeros_like(X))
    local_tval = np.matrix(np.zeros_like(X))

    for j in range(K):
      # X_jを要素とする対角行列の作成
      Xj = np.asarray(X[:, j]).flatten()
      diag_Xj = np.matrix(np.diag(Xj))
      # R_j
      R_j = np.matrix(R_list[j])

      # パラメータ推定値の分散共分散行列の対角要素から標準誤差を求める
      C = np.linalg.inv(diag_Xj) @ R_j
      CCT = C @ C.T
      var_beta_j = sigma_hat**2 * np.diag(CCT)
      std_beta_j = np.sqrt(var_beta_j)

      # t値を求める
      beta_hat_j = np.asarray(beta_hat[:,j]).flatten()
      tval_j = beta_hat_j / std_beta_j

      # std,tvalに格納
      local_std[:, j] = std_beta_j.reshape(-1, 1)
      local_tval[:, j] = tval_j.reshape(-1, 1)

    return local_std, local_tval


  def fit(self):
    self.beta_hat = self.get_beta_hat()
    self.S, self.R_list = self.get_S_Rlist()
    self.v1 = get_v1(self.S)
    self.df = self.N - self.v1
    self.y_hat = get_y_hat(self.y, self.S)
    self.resid = get_resid(self.y, self.y_hat)
    self.rss = get_rss(self.resid)
    self.sigma2 = get_sigma2(self.rss, self.N, self.v1)
    self.sigma_hat = np.sqrt(self.sigma2)
    self.mll = get_mll(self.y, self.y_hat, self.rss, self.N)
    self.aic = get_aic(self.mll, self.v1+1)
    self.aicc = get_aicc(self.mll, self.N, self.v1+1)
    self.bic = get_bic(self.mll, self.N, self.v1+1)
    self.R2, self.adjR2 = get_R2(self.y, self.rss, self.N, self.v1+1)
    self.local_std, self.local_tval = self.get_local_std_and_tval()  # 追加部分
    self.adj_alpha = self.get_adj_alpha()  # 追加部分
    self.adj_tval = self.get_adj_tval()  # 追加部分

In [15]:
res = MGWR(y, X, coords, opt_list)
res.fit()

In [16]:
print(res.local_tval)

[[-2.42612165  2.41357244 -0.57236925 -5.2139234 ]
 [-3.08578219  2.1765894  -0.33875446 -5.12844925]
 [-2.7082607   2.28892869 -0.45913712 -5.20561331]
 [-1.79639248  1.67839468 -0.33398642 -4.22346053]
 [ 1.99799841  6.91590217 -2.0740209  -4.26225848]
 [ 2.86728131  8.03334032 -1.36121739 -4.40265519]
 [ 3.1473972   8.11392106 -1.47638843 -4.42733199]
 [ 1.58787852  8.21908664 -0.83550011 -4.37366273]
 [-2.48732562  2.41490715 -0.54879141 -5.01124639]
 [-3.01070018  2.09106812 -0.32788887 -5.00815984]
 [ 1.15216843  5.95104764 -1.79148332 -4.19522575]
 [-0.3358969   3.36894378 -1.23673802 -4.69777019]
 [-2.58914295  2.11636281 -0.3339112  -5.25257323]
 [-2.9270513   1.61718815 -0.22729276 -4.86402661]
 [-1.49965438  2.53985162 -0.82695792 -5.19594507]
 [-0.88019351  2.93938347 -1.1658536  -5.08724818]
 [ 0.74617453  4.46682988 -1.77850006 -4.71543767]
 [ 2.58364532  7.19704854 -1.92720791 -4.46154077]
 [-1.45227887  1.88077231 -0.41304583 -4.03572799]
 [-2.53322806  2.06116078 -0.27